# Analyse exploratoire complète des données (EDA)

**Auteur:** Louis Vanacker

**Date:** 7 janvier 2026

**Objectif:** Réaliser une analyse exploratoire complète du jeu de données Students Performance in Exams.

## Table des matières
1. [Chargement et aperçu des données](#1)
2. [Analyse univariée](#2)
3. [Analyse bivariée](#3)
4. [Analyse multivariée](#4)
5. [Feature Engineering](#5)
6. [Conclusions et insights](#6)

<a id='1'></a>
## 1. Chargement et aperçu des données

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration du style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print('Bibliothèques importées avec succès')

In [ ]:
# Chargement des données
url = "https://raw.githubusercontent.com/Dorsumsellae/Programmation-avancee-Projet-d-examen-Students-Performance-in-Exams/main/data/raw/StudentsPerformance.csv"
df = pd.read_csv(url)

print(f'Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes')
print('\nAperçu des 5 premières lignes :')
df.head()

In [ ]:
# Informations sur le dataset
print('=== INFORMATIONS SUR LE DATASET ===')
df.info()

print('\n=== STATISTIQUES DESCRIPTIVES ===')
df.describe()

In [ ]:
# Vérification de la qualité des données
print('=== QUALITÉ DES DONNÉES ===')
print(f'\nValeurs manquantes : {df.isnull().sum().sum()}')
print(f'Doublons : {df.duplicated().sum()}')
print('\nTypes de données :')
print(df.dtypes)
print('\nValeurs uniques par colonne :')
for col in df.columns:
    print(f'  - {col}: {df[col].nunique()} valeurs')

<a id='2'></a>
## 2. Analyse univariée

### 2.1 Variables catégorielles

In [ ]:
# Liste des variables
categorical_cols = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
score_cols = ['math score', 'reading score', 'writing score']

print('=== ANALYSE DES VARIABLES CATÉGORIELLES ===')

for col in categorical_cols:
    print(f'\n{col.upper()}:')
    print(df[col].value_counts())
    print(f'\nDistribution en % :')
    print(df[col].value_counts(normalize=True).mul(100).round(1))

In [ ]:
# Visualisation des variables catégorielles
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(categorical_cols):
    counts = df[col].value_counts()
    axes[i].bar(range(len(counts)), counts.values, color='steelblue', alpha=0.7, edgecolor='black')
    axes[i].set_xticks(range(len(counts)))
    axes[i].set_xticklabels(counts.index, rotation=45, ha='right')
    axes[i].set_title(f'Distribution de {col}', fontsize=12, fontweight='bold')
    axes[i].set_ylabel('Effectif', fontsize=11)
    axes[i].grid(axis='y', alpha=0.3)
    
    for j, v in enumerate(counts.values):
        axes[i].text(j, v + 10, f'{v}\n({v/len(df)*100:.1f}%)', 
                    ha='center', va='bottom', fontsize=9)

fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

### 2.2 Variables numériques (Scores)

In [ ]:
print('=== STATISTIQUES DES SCORES ===')
print(df[score_cols].describe())

print('\n=== STATISTIQUES DÉTAILLÉES PAR SCORE ===')

for col in score_cols:
    print(f'\n{col.upper()}:')
    print(f'  Moyenne     : {df[col].mean():.2f}')
    print(f'  Médiane     : {df[col].median():.2f}')
    print(f'  Mode        : {df[col].mode()[0]}')
    print(f'  Écart-type  : {df[col].std():.2f}')
    print(f'  Variance    : {df[col].var():.2f}')
    print(f'  Skewness    : {df[col].skew():.2f}')
    print(f'  Kurtosis    : {df[col].kurtosis():.2f}')
    print(f'  Min         : {df[col].min()}')
    print(f'  Max         : {df[col].max()}')
    print(f'  Étendue     : {df[col].max() - df[col].min()}')

In [ ]:
# Distributions des scores avec statistiques
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    axes[i].hist(df[col], bins=20, edgecolor='black', alpha=0.7, color='skyblue')
    axes[i].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Moyenne: {df[col].mean():.1f}')
    axes[i].axvline(df[col].median(), color='green', linestyle='--', linewidth=2,
                   label=f'Médiane: {df[col].median():.1f}')
    axes[i].set_title(f'Distribution de {col}', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Score', fontsize=12)
    axes[i].set_ylabel('Fréquence', fontsize=12)
    axes[i].legend(fontsize=10)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots pour détecter les outliers
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    bp = axes[i].boxplot(df[col], vert=True, patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][0].set_edgecolor('black')
    axes[i].set_title(f'{col} - Boîte à moustaches', fontsize=14, fontweight='bold')
    axes[i].set_ylabel('Score', fontsize=12)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Détection des valeurs aberrantes (méthode IQR)
print('=== DÉTECTION DES VALEURS ABERRANTES (MÉTHODE IQR) ===')

for col in score_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    
    print(f'\n{col.upper()}:')
    print(f'  Q1 (25%)            : {Q1:.2f}')
    print(f'  Q3 (75%)            : {Q3:.2f}')
    print(f'  IQR                 : {IQR:.2f}')
    print(f'  Borne inférieure    : {lower_bound:.2f}')
    print(f'  Borne supérieure    : {upper_bound:.2f}')
    print(f'  Valeurs aberrantes  : {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)')

<a id='3'></a>
## 3. Analyse bivariée

### 3.1 Corrélations entre les scores

In [ ]:
# Matrice de corrélation
correlation_matrix = df[score_cols].corr()

print('=== MATRICE DE CORRÉLATION ENTRE LES SCORES ===')
print(correlation_matrix)

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', square=True, linewidths=1, cbar_kws={'label': 'Corrélation'})
plt.title('Heatmap des corrélations entre les scores', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Pairplot des scores
sns.pairplot(df[score_cols], diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Pairplot des scores', fontsize=16, fontweight='bold', y=1.01)
plt.show()

### 3.2 Impact du genre sur les scores

In [ ]:
# Statistiques par genre
print('=== SCORES MOYENS PAR GENRE ===')
print(df.groupby('gender')[score_cols].mean())

print('\n=== ÉCART-TYPE PAR GENRE ===')
print(df.groupby('gender')[score_cols].std())

In [ ]:
# Visualisation par genre
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    sns.boxplot(data=df, x='gender', y=col, ax=axes[i], palette='Set2')
    axes[i].set_title(f'{col} par genre', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Genre', fontsize=12)
    axes[i].set_ylabel('Score', fontsize=12)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 Impact du niveau d'éducation des parents

In [ ]:
# Statistiques par niveau d'éducation
print('=== SCORES MOYENS PAR NIVEAU D\'ÉDUCATION DES PARENTS ===')
print(df.groupby('parental level of education')[score_cols].mean().round(2))

In [ ]:
# Visualisation
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for i, col in enumerate(score_cols):
    sns.boxplot(data=df, x='parental level of education', y=col, ax=axes[i], palette='Set3')
    axes[i].set_title(f'{col} par niveau d\'éducation des parents', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Niveau d\'éducation', fontsize=12)
    axes[i].set_ylabel('Score', fontsize=12)
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 3.4 Impact du type de déjeuner (lunch)

In [ ]:
# Statistiques par type de lunch
print('=== SCORES MOYENS PAR TYPE DE DÉJEUNER ===')
print(df.groupby('lunch')[score_cols].mean())

print('\n=== DIFFÉRENCE ENTRE STANDARD ET FREE/REDUCED ===')
for col in score_cols:
    diff = df[df['lunch']=='standard'][col].mean() - df[df['lunch']=='free/reduced'][col].mean()
    print(f'{col}: {diff:.2f} points')

In [ ]:
# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    sns.violinplot(data=df, x='lunch', y=col, ax=axes[i], palette='muted')
    axes[i].set_title(f'{col} par type de déjeuner', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Type de déjeuner', fontsize=12)
    axes[i].set_ylabel('Score', fontsize=12)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 3.5 Impact du cours de préparation aux tests

In [ ]:
# Statistiques par préparation
print('=== SCORES MOYENS PAR COURS DE PRÉPARATION ===')
print(df.groupby('test preparation course')[score_cols].mean())

print('\n=== GAIN MOYEN AVEC LE COURS DE PRÉPARATION ===')
for col in score_cols:
    gain = df[df['test preparation course']=='completed'][col].mean() - df[df['test preparation course']=='none'][col].mean()
    print(f'{col}: +{gain:.2f} points')

In [ ]:
# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    sns.boxplot(data=df, x='test preparation course', y=col, ax=axes[i], palette='pastel')
    axes[i].set_title(f'{col} par cours de préparation', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Cours de préparation', fontsize=12)
    axes[i].set_ylabel('Score', fontsize=12)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 3.6 Impact de l'origine ethnique

In [ ]:
# Statistiques par groupe ethnique
print('=== SCORES MOYENS PAR GROUPE ETHNIQUE ===')
print(df.groupby('race/ethnicity')[score_cols].mean().round(2))

In [ ]:
# Visualisation
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for i, col in enumerate(score_cols):
    sns.boxplot(data=df, x='race/ethnicity', y=col, ax=axes[i], palette='husl')
    axes[i].set_title(f'{col} par groupe ethnique', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Groupe ethnique', fontsize=12)
    axes[i].set_ylabel('Score', fontsize=12)
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

<a id='4'></a>
## 4. Analyse multivariée

In [ ]:
# Analyse combinée : Genre + Cours de préparation
print('=== IMPACT COMBINÉ : GENRE + COURS DE PRÉPARATION ===')
print(df.groupby(['gender', 'test preparation course'])[score_cols].mean().round(2))

In [ ]:
# Visualisation combinée
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    sns.barplot(data=df, x='gender', y=col, hue='test preparation course', ax=axes[i], palette='Set1')
    axes[i].set_title(f'{col} - Genre x Préparation', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Genre', fontsize=12)
    axes[i].set_ylabel('Score moyen', fontsize=12)
    axes[i].legend(title='Préparation')
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyse combinée : Lunch + Cours de préparation
print('=== IMPACT COMBINÉ : TYPE DE DÉJEUNER + COURS DE PRÉPARATION ===')
print(df.groupby(['lunch', 'test preparation course'])[score_cols].mean().round(2))

In [ ]:
# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(score_cols):
    sns.barplot(data=df, x='lunch', y=col, hue='test preparation course', ax=axes[i], palette='Set2')
    axes[i].set_title(f'{col} - Déjeuner x Préparation', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Type de déjeuner', fontsize=12)
    axes[i].set_ylabel('Score moyen', fontsize=12)
    axes[i].legend(title='Préparation')
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

<a id='5'></a>
## 5. Feature Engineering

In [ ]:
# Création de nouvelles features
df['total_score'] = df['math score'] + df['reading score'] + df['writing score']
df['average_score'] = df['total_score'] / 3

# Catégories de performance
def categorize_performance(score):
    if score >= 80:
        return 'Excellent'
    elif score >= 70:
        return 'Bien'
    elif score >= 60:
        return 'Moyen'
    elif score >= 50:
        return 'Passable'
    else:
        return 'Faible'

df['performance_category'] = df['average_score'].apply(categorize_performance)

# Score le plus élevé
df['best_subject'] = df[score_cols].idxmax(axis=1).str.replace(' score', '')

# Score le plus faible
df['worst_subject'] = df[score_cols].idxmin(axis=1).str.replace(' score', '')

print('Nouvelles features créées :')
print('  - total_score')
print('  - average_score')
print('  - performance_category')
print('  - best_subject')
print('  - worst_subject')
print('\nAperçu des nouvelles colonnes :')
df[['total_score', 'average_score', 'performance_category', 'best_subject', 'worst_subject']].head(10)

In [ ]:
# Distribution des catégories de performance
print('=== DISTRIBUTION DES CATÉGORIES DE PERFORMANCE ===')
print(df['performance_category'].value_counts())
print('\nEn pourcentage :')
print(df['performance_category'].value_counts(normalize=True).mul(100).round(1))

In [ ]:
# Visualisation des nouvelles features
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Score total
axes[0, 0].hist(df['total_score'], bins=30, edgecolor='black', alpha=0.7, color='lightcoral')
axes[0, 0].axvline(df['total_score'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Moyenne: {df["total_score"].mean():.1f}')
axes[0, 0].set_title('Distribution du score total', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Score total', fontsize=12)
axes[0, 0].set_ylabel('Fréquence', fontsize=12)
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# Catégories de performance
perf_counts = df['performance_category'].value_counts()
colors = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c', '#95a5a6']
axes[0, 1].bar(range(len(perf_counts)), perf_counts.values, color=colors, edgecolor='black')
axes[0, 1].set_xticks(range(len(perf_counts)))
axes[0, 1].set_xticklabels(perf_counts.index, rotation=45)
axes[0, 1].set_title('Distribution des catégories de performance', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Effectif', fontsize=12)
axes[0, 1].grid(axis='y', alpha=0.3)

# Meilleure matière
best_counts = df['best_subject'].value_counts()
axes[1, 0].bar(range(len(best_counts)), best_counts.values, color='steelblue', edgecolor='black')
axes[1, 0].set_xticks(range(len(best_counts)))
axes[1, 0].set_xticklabels(best_counts.index, rotation=45)
axes[1, 0].set_title('Matière avec le meilleur score', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Nombre d\'étudiants', fontsize=12)
axes[1, 0].grid(axis='y', alpha=0.3)

# Pire matière
worst_counts = df['worst_subject'].value_counts()
axes[1, 1].bar(range(len(worst_counts)), worst_counts.values, color='coral', edgecolor='black')
axes[1, 1].set_xticks(range(len(worst_counts)))
axes[1, 1].set_xticklabels(worst_counts.index, rotation=45)
axes[1, 1].set_title('Matière avec le score le plus faible', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Nombre d\'étudiants', fontsize=12)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

<a id='6'></a>
## 6. Conclusions et insights

### 6.1 Résumé des découvertes principales

In [ ]:
print('='*80)
print('RÉSUMÉ EXÉCUTIF - INSIGHTS CLÉS')
print('='*80)

print('\n1. QUALITÉ DES DONNÉES')
print(f'   • Dataset propre : {df.isnull().sum().sum()} valeurs manquantes, {df.duplicated().sum()} doublons')
print(f'   • {len(df)} étudiants, {len(categorical_cols)} variables catégorielles, {len(score_cols)} scores')

print('\n2. SCORES MOYENS')
for col in score_cols:
    print(f'   • {col}: {df[col].mean():.2f} ± {df[col].std():.2f}')
print(f'   • Les maths sont la matière la plus faible en moyenne')

print('\n3. IMPACT DU GENRE')
print(f'   • Femmes meilleures en lecture/écriture')
print(f'   • Hommes légèrement meilleurs en maths')
female_math = df[df['gender']=='female']['math score'].mean()
male_math = df[df['gender']=='male']['math score'].mean()
print(f'   • Écart maths: {abs(male_math - female_math):.2f} points')

print('\n4. IMPACT DE L\'ÉDUCATION PARENTALE')
edu_impact = df.groupby('parental level of education')['average_score'].mean()
print(f'   • Meilleur niveau: {edu_impact.idxmax()} ({edu_impact.max():.2f})')
print(f'   • Niveau le plus bas: {edu_impact.idxmin()} ({edu_impact.min():.2f})')
print(f'   • Différence: {edu_impact.max() - edu_impact.min():.2f} points')

print('\n5. IMPACT DU TYPE DE DÉJEUNER')
standard_avg = df[df['lunch']=='standard']['average_score'].mean()
reduced_avg = df[df['lunch']=='free/reduced']['average_score'].mean()
print(f'   • Standard: {standard_avg:.2f}')
print(f'   • Free/Reduced: {reduced_avg:.2f}')
print(f'   • Différence: {standard_avg - reduced_avg:.2f} points')
print(f'   • Indicateur socio-économique fort')

print('\n6. IMPACT DU COURS DE PRÉPARATION')
prep_yes = df[df['test preparation course']=='completed']['average_score'].mean()
prep_no = df[df['test preparation course']=='none']['average_score'].mean()
print(f'   • Avec préparation: {prep_yes:.2f}')
print(f'   • Sans préparation: {prep_no:.2f}')
print(f'   • Gain moyen: +{prep_yes - prep_no:.2f} points')

print('\n7. CORRÉLATIONS')
print(f'   • Math-Reading: {df["math score"].corr(df["reading score"]):.3f}')
print(f'   • Math-Writing: {df["math score"].corr(df["writing score"]):.3f}')
print(f'   • Reading-Writing: {df["reading score"].corr(df["writing score"]):.3f}')
print(f'   • Reading et Writing très corrélés')

print('\n8. PERFORMANCE GLOBALE')
print(df['performance_category'].value_counts())
excellent_pct = (df['performance_category']=='Excellent').sum() / len(df) * 100
print(f'   • {excellent_pct:.1f}% des étudiants en catégorie "Excellent"')

print('\n' + '='*80)

### 6.2 Recommandations pour la modélisation

**Variables importantes à considérer:**
1. test preparation course - Impact fort (environ +5 points en moyenne)
2. lunch - Indicateur socio-économique puissant
3. parental level of education - Corrélé avec la réussite
4. gender - Impact différencié selon la matière
5. race/ethnicity - À utiliser avec précaution

**Features engineered utiles:**
- average_score - Pour prédiction globale
- total_score - Peut capturer des patterns
- Interactions entre variables (genre × préparation, lunch × préparation)

**Encodage nécessaire:**
- One-Hot Encoding pour les variables catégorielles
- Standardisation des scores si nécessaire

**Algorithmes recommandés:**
- Régression linéaire (baseline)
- Random Forest (capture interactions)
- XGBoost (performance optimale)
- Neural Networks (si assez de données)